# Question Generator (QG) Model Using HuggingFace T5

## Install and Import Needed Library

In [ ]:
!pip install transformers datasets sentencepiece

In [ ]:
!pip install transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=806845b3c46655f96a8b89d10fb6255a6033e511b644b87017646ee65df3d28e
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [2]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e1346d2080fc6469f5da7a8819631ab024523637d89506228c49e204a5805495
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [3]:
from google.colab import files
import pandas as pd
import random
from datasets import Dataset
import torch
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from rouge_score import rouge_scorer

## Data Loading

In [4]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"meilanikizana","key":"230db92153864ff31f16103462e6c659"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d mintupsidup/squad-variated-indo
!unzip squad-variated-indo.zip

Dataset URL: https://www.kaggle.com/datasets/mintupsidup/squad-variated-indo
License(s): apache-2.0
 86% 88.0M/103M [00:00<00:00, 919MB/s]
100% 103M/103M [00:00<00:00, 922MB/s] 
Archive:  squad-variated-indo.zip
  inflating: dev-v2.0.json           
  inflating: glove_50d_Indo.txt      
  inflating: train-v2.0.json         
  inflating: train_df_variated_new.csv  
  inflating: valid_df_variated_new.csv  


In [7]:
train = pd.read_json('train-v2.0.json', orient='column')
dev = pd.read_json('dev-v2.0.json', orient='column')

init_df = pd.concat([train, dev], ignore_index=True)
init_df

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'context'..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'context': '..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...
...,...,...
472,v2.0,"{'title': 'Islamism', 'paragraphs': [{'context..."
473,v2.0,"{'title': 'Imperialism', 'paragraphs': [{'cont..."
474,v2.0,"{'title': 'Warsaw', 'paragraphs': [{'context':..."
475,v2.0,"{'title': 'French_and_Indian_War', 'paragraphs..."


In [8]:
init_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   version  477 non-null    object
 1   data     477 non-null    object
dtypes: object(2)
memory usage: 7.6+ KB


## Data Preparation

### Extract Features

In [9]:
contexts, questions, answers = [], [], []

for data in init_df['data']:
    for para in data['paragraphs']:
        context = para['context']
        for all_questions in para['qas']:
            if not all_questions.get("is_impossible", False):
                question = all_questions['question']
                answer_list = all_questions['answers']
                if len(answer_list) > 0:
                    answer = answer_list[0]['text']
                    questions.append(question)
                    contexts.append(context)
                    answers.append(answer)

print(f"Total data pertanyaan: {len(questions)}")

Total data pertanyaan: 92749


In [10]:
df = pd.DataFrame({
    "context": contexts,
    "question": questions,
    "answer": answers
})

df

,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce mulai menjadi populer?,pada akhir 1990-an
1,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Daerah mana saja yang bersaing dengan Beyonce ...,menyanyi dan
2,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce meninggalkan Destiny's Child dan...,2003
3,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Di kota dan negara mana Beyonce tumbuh?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Pada dekade mana Beyonce menjadi terkenal?,akhir 1990-an
...,...,...,...
92744,"Pound-force memiliki mitra metrik, kurang umum...",Apa istilah metrik yang kurang digunakan dari ...,kilogram-force
92745,"Pound-force memiliki mitra metrik, kurang umum...",Apa daya kilogram kadang-kadang direffer ke se...,kilopond
92746,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan unit massa dalam sis...,siput
92747,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan istilah unit kekuata...,kip


### Handling Missing Values and Duplicated Data

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92749 entries, 0 to 92748
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   92749 non-null  object
 1   question  92749 non-null  object
 2   answer    92749 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [12]:
print("Missing values:\n", df.isnull().sum())
print("\nDuplikasi baris:", df.duplicated().sum())

Missing values:
 context     0
question    0
answer      0
dtype: int64

Duplikasi baris: 55


In [13]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce mulai menjadi populer?,pada akhir 1990-an
1,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Daerah mana saja yang bersaing dengan Beyonce ...,menyanyi dan
2,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Kapan Beyonce meninggalkan Destiny's Child dan...,2003
3,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Di kota dan negara mana Beyonce tumbuh?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/bijanse / bee...,Pada dekade mana Beyonce menjadi terkenal?,akhir 1990-an
...,...,...,...
92689,"Pound-force memiliki mitra metrik, kurang umum...",Apa istilah metrik yang kurang digunakan dari ...,kilogram-force
92690,"Pound-force memiliki mitra metrik, kurang umum...",Apa daya kilogram kadang-kadang direffer ke se...,kilopond
92691,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan unit massa dalam sis...,siput
92692,"Pound-force memiliki mitra metrik, kurang umum...",Apa yang jarang digunakan istilah unit kekuata...,kip


In [14]:
print("Duplikasi baris:", df.duplicated().sum())

Duplikasi baris: 0


### Data Sampling, Convert to HuggingFace Dataset & Split to Train and Test

In [84]:
sample_size = 2000
df_sample = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

contexts_list = df_sample['context'].tolist()
questions_list = df_sample['question'].tolist()
answers_list = df_sample['answer'].tolist()

raw_dataset = Dataset.from_dict({
    "context": contexts_list,
    "question": questions_list,
    "answer": answers_list
})

raw_dataset = raw_dataset.train_test_split(test_size=0.2)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 400
    })
})

### Preprocessing and Tokenization

In [85]:
tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

def preprocess(text):
    input_text = f"generate question: context: {text['context']} answer: {text['answer']}"
    target_text = text["question"]

    model_inputs = tokenizer(
        input_text, max_length=512, padding="max_length", truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_text, max_length=64, padding="max_length", truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = raw_dataset.map(preprocess, batched=False)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

## Load and Configure T5 Model

In [90]:
t5_model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

In [91]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()

Step,Training Loss
100,1.074300
200,0.612100
300,0.613000
400,0.586900


TrainOutput(global_step=400, training_loss=0.7215832233428955, metrics={'train_runtime': 202.1154, 'train_samples_per_second': 7.916, 'train_steps_per_second': 1.979, 'total_flos': 974332624896000.0, 'train_loss': 0.7215832233428955, 'epoch': 1.0})

## Load Indonesian Named Entity Recognition (NER) Model

In [93]:
ner_tokenizer = AutoTokenizer.from_pretrained("cahya/bert-base-indonesian-NER")
ner_model = AutoModelForTokenClassification.from_pretrained("cahya/bert-base-indonesian-NER")

ner_pipe = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, grouped_entities=True)

Some weights of the model checkpoint at cahya/bert-base-indonesian-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


## Functions

In [94]:
def extract_entities(text, max_entities=5):
    entities = ner_pipe(text)
    answers = []
    for entity in entities:
        if entity['entity_group'] in ['PER', 'LOC', 'DAT']:
            word = entity['word'].replace('##', '').strip()
            if word not in answers:
                answers.append(word)
        if len(answers) >= max_entities:
            break
    return answers

In [95]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t5_model.to(device)

def get_sentence_with_answer(text, answer):
    sentences = re.split(r'(?<=[.!?]) +', text)
    for sentence in sentences:
        if answer in sentence:
            return sentence
    return text

def generate_question_from_input(input_text):
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    outputs = t5_model.generate(
        inputs["input_ids"],
        max_length=64,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_questions(text, num_questions=5):
    answers = extract_entities(text, max_entities=num_questions)
    questions = []

    if answers:
        for ans in answers:
            context = get_sentence_with_answer(text, ans)
            input_text = f"generate question: context: {context} answer: {ans}"
            question = generate_question_from_input(input_text)
            questions.append(question)
            remaining_questions = num_questions - len(questions)
        if remaining_questions > 0:
            for i in range(remaining_questions):
                input_text = f"generate question: context: {text}"
                question = generate_question_from_input(input_text)
                questions.append(question)
    else:
        for _ in range(num_questions):
            input_text = f"generate question: context: {text}"
            question = generate_question_from_input(input_text)
            questions.append(question)

    return questions

## Inference Process

In [96]:
text = "Hari lahir Pancasila jatuh pada tanggal 1 Juni yang ditandai oleh pidato yang dilakukan oleh Presiden pertama Indonesia Ir Soekarno pada tanggal 1 Juni 1945 dalam sidang Dokuritsu Junbi Cosakai (Badan Penyelidik Usaha Persiapan Kemerdekaan). Dalam pidatonya pertama kali mengemukakan konsep awal Pancasila yang menjadi dasar negara Indonesia. Adapun sejarahnya berawal dari kekalahan Jepang pada perang pasifik, mereka kemudian berusaha mendapatkan hati masyarakat dengan menjanjikan kemerdekaan kepada Indonesia dan membentuk sebuah Lembaga yang tugasnya untuk mempersiapkan hal tersebut. Lembaga ini dinamai Dokuritsu Junbi Cosakai. Pada sidang pertamanya di tanggal 29 Mei 1945 yang diadakan di Gedung Chuo Sangi In (sekarang Gedung Pancasila), para anggota membahas mengenai tema dasar negara. Sidang berjalan sekitar hampir 5 hari, kemudian pada tanggal 1 Juni 1945, Soekarno menyampaikan ide serta gagasannya terkait dasar negara Indonesia, yang dinamai “Pancasila”. Panca artinya lima, sedangkan sila artinya prinsip atau asas."
questions = generate_questions(text, num_questions=5)

for i, q in enumerate(questions, 1):
    print(f"{i}. {q}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1. Tahun berapa persidangan Dokuritsu Junbi membuat sebuah kesimpulan mengenai Pancasila?
2. Kapan Soekarno menyampaikan ide tentang dasar negara?
3. Kapan pidato Presiden pertama Soekarno memberikan ide penting kepada Pancasila?
4. Kapan sidang Dokuritsu Junbi Cosakai berakhir?
5. Kapan Soeserto dan Dokuritsu?


## Evaluation

In [97]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

### Data Train 500

In [98]:
sample_data = raw_dataset['train'][500]
context_500 = sample_data['context']
reference_question_500 = sample_data['question']

print("Konteks:\n", context_500)
print("\nPertanyaan:\n", reference_question_500)

Konteks:
 Paralel terhadap perkembangan militer muncul juga terus-menerus lebih rumit kode chivalric perilaku untuk kelas prajurit. etos baru ini dapat dilihat sebagai respon terhadap berkurangnya peran militer dari aristokrasi, dan secara bertahap menjadi hampir sepenuhnya terpisah dari asal militernya. Semangat ksatria diberikan ekspresi melalui jenis baru chivalric order; yang pertama adalah Orde St. George, didirikan oleh Charles I dari Hongaria pada tahun 1325, sementara yang paling dikenal mungkin Orde Inggris Garter, didirikan oleh Edward III pada tahun 1348.

Pertanyaan:
 Pada tahun berapa Order of the Garter didirikan?


In [99]:
questions_500 = generate_questions(context_500, num_questions=5)
generated_question_500 = []

for i, q in enumerate(questions_500, 1):
    generated_question_500.append(q)
    print(f"{i}. {q}")

1. Apa yang disebut sebagai chivalric yang menyerang.
2. Kapan George didirikan Charles I dari Hongaria?
3. Siapa yang memberikan definisi bagaimana chivalric-sminse?
4. Siapa yang mendirikan George pada tahun 1325?
5. Istilah apa yang digunakan untuk apa?


In [100]:
for i, q in enumerate(generated_question_500, 1):
    scores = scorer.score(reference_question_500, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
2. {'rouge1': Score(precision=0.14285714285714285, recall=0.125, fmeasure=0.13333333333333333), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.14285714285714285, recall=0.125, fmeasure=0.13333333333333333)}
3. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
4. {'rouge1': Score(precision=0.2857142857142857, recall=0.25, fmeasure=0.26666666666666666), 'rouge2': Score(precision=0.16666666666666666, recall=0.14285714285714285, fmeasure=0.15384615384615383), 'rougeL': Score(precision=0.2857142857142857, recall=0.25, fmeasure=0.26666666666666666)}
5. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fme

### Data Train 250

In [101]:
sample_data = raw_dataset['train'][250]
context_250 = sample_data['context']
reference_question_250 = sample_data['question']

print("Konteks:\n", context_250)
print("\nPertanyaan:\n", reference_question_250)

Konteks:
 Kenneth Gergen merumuskan klasifikasi tambahan, yang mencakup manipulator strategis, kepribadian pastiche, dan relasi diri. Manipulator strategis adalah orang yang mulai menganggap semua indra identitas hanya sebagai latihan bermain peran, dan secara bertahap menjadi terasing dari dirinya sendiri. Kepribadian pasdiche meninggalkan semua aspirasi menuju identitas yang benar atau "penting," daripada melihat interaksi sosial sebagai kesempatan untuk bermain, dan oleh karena itu menjadi, peran mereka bermain. Akhirnya, diri relasional adalah sudut pandang orang - orang yang meninggalkan semua rasa diri yang eksklusif, dan memandang semua rasa identitas dalam hal keterlibatan sosial dengan orang lain. Bagi Gergen, strategi ini saling mengikuti dalam tahapan, dan mereka dikaitkan dengan meningkatnya popularitas budaya postmodern dan munculnya teknologi telekomunikasi.

Pertanyaan:
 Dalam klasifikasi apa seseorang yang menyerahkan diri eksklusif mereka dan mendefinisikan dirinya han

In [102]:
questions_250 = generate_questions(context_250, num_questions=5)
generated_question_250 = []

for i, q in enumerate(questions_250, 1):
    generated_question_250.append(q)
    print(f"{i}. {q}")

1. Bagaimana Gergen merumuskannya?
2. Apa namanya dia meninggalkan aspirasi untuk menjadi 'penting'?
3. Siapa yang merumuskan manipulator?
4. Apa nama manipulatif?
5. Apa klasifikasi ini?


In [103]:
for i, q in enumerate(generated_question_250, 1):
    scores = scorer.score(reference_question_250, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
2. {'rouge1': Score(precision=0.125, recall=0.058823529411764705, fmeasure=0.07999999999999999), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.125, recall=0.058823529411764705, fmeasure=0.07999999999999999)}
3. {'rouge1': Score(precision=0.25, recall=0.058823529411764705, fmeasure=0.09523809523809523), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.25, recall=0.058823529411764705, fmeasure=0.09523809523809523)}
4. {'rouge1': Score(precision=0.3333333333333333, recall=0.058823529411764705, fmeasure=0.1), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.3333333333333333, recall=0.058823529411764705, fmeasure=0.1)}
5. {'rouge1': Score(precision=0.6666666666666666, recall=0.11764705882352941, fmeasure

### Data Train 150

In [104]:
sample_data = raw_dataset['train'][150]
context_150 = sample_data['context']
reference_question_150 = sample_data['question']

print("Konteks:\n", context_150)
print("\nPertanyaan:\n", reference_question_150)

Konteks:
 Pada tahun 1565, pangeran kuat Rinbung digulingkan oleh salah satu menteri mereka sendiri, Karma Tseten yang bergaya dirinya sebagai Tsangpa, "salah satu dari Tsang", dan mendirikan basis kekuasaan di Shigatse. Penerus kedua dari raja Tsang pertama ini, Karma Phuntsok Namgyal, mengambil alih seluruh Central Tibet (#-Tsang), yang berkuasa dari 16111621. Meskipun demikian, para pemimpin Lhasa masih mengklaim kesetiaan mereka kepada Phagmodru serta Gelug, sementara raja Tsang bersekutu dengan Karmapa. Ketegangan meningkat antara penguasa nasionalistik-Tsang dan Mongol yang menjaga mereka Mongol Dalai Lama di Lhasa. Dalai Lama keempat menolak untuk memberikan penonton kepada raja #-Tsang, yang memicu konflik sebagai yang terakhir mulai menyerang biara Gelug. Chen menulis tentang spekulasi atas kematian misterius Dalai Lama keempat dan plot raja #-Tsang untuk memiliki dia dibunuh untuk "menghukum" dia dengan penyakit, meskipun Chen menulis bahwa pembunuhan kemungkinan besar hasil 

In [105]:
questions_150 = generate_questions(context_150, num_questions=5)
generated_question_150 = []

for i, q in enumerate(questions_150, 1):
    generated_question_150.append(q)
    print(f"{i}. {q}")

1. Pada tahun 1565, pangeran kuat Rinbung digulingkan oleh salah satu menterinya sendiri, Karma Tseten.
2. Apa yang mendorong kekuatan Rinbung digulingkan?
3. Siapa yang memerintah orang lain yang disebut sebagai Tsangpa?
4. Siapa yang disebut oleh Dalai Lama?
5. Apa pemimpin Tibet yang sebelumnya menguasai seluruh Central Tibet?


In [106]:
for i, q in enumerate(generated_question_150, 1):
    scores = scorer.score(reference_question_150, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
2. {'rouge1': Score(precision=0.16666666666666666, recall=0.14285714285714285, fmeasure=0.15384615384615383), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.16666666666666666, recall=0.14285714285714285, fmeasure=0.15384615384615383)}
3. {'rouge1': Score(precision=0.2222222222222222, recall=0.2857142857142857, fmeasure=0.25), 'rouge2': Score(precision=0.125, recall=0.16666666666666666, fmeasure=0.14285714285714288), 'rougeL': Score(precision=0.2222222222222222, recall=0.2857142857142857, fmeasure=0.25)}
4. {'rouge1': Score(precision=0.3333333333333333, recall=0.2857142857142857, fmeasure=0.30769230769230765), 'rouge2': Score(precision=0.2, recall=0.16666666666666666, fmeasure=0.1818181818181818), 'rougeL': Score(precision=0.3333333333333333, recall=0.2857142857142857, fmea

### Data Train 50

In [107]:
sample_data = raw_dataset['train'][50]
context_50 = sample_data['context']
reference_question_50 = sample_data['question']

print("Konteks:\n", context_50)
print("\nPertanyaan:\n", reference_question_50)

Konteks:
 Penyebab yang tepat dan motivasi untuk konflik militer Roma dan perluasan selama republik ini tunduk pada perdebatan luas. Meskipun mereka dapat dipandang dimotivasi oleh agresi dan imperialisme yang terang - terangan, para sejarawan biasanya memiliki pandangan yang jauh lebih bernuansa. Mereka berpendapat bahwa ekspansi Roma didorong oleh faktor pertahanan jangka pendek dan antar negara (yaitu, hubungan dengan negara-kota dan kerajaan-kerajaan di luar hegemoni Roma), dan kontinjensi baru yang dibuat keputusan-keputusan ini. Dalam sejarah awal, sebagai Roma berhasil mempertahankan diri terhadap ancaman asing di pusat dan kemudian Italia utara, negara-kota tetangga mencari perlindungan aliansi Romawi akan membawa. Dengan demikian, Roma Republik awal bukanlah sebuah "empire" atau "state" dalam arti modern, tetapi aliansi negara-kota independen (sama dengan hegemoni Yunani dari periode yang sama) dengan berbagai derajat kemerdekaan sejati (yang berubah dari waktu ke waktu) terli

In [108]:
questions_50 = generate_questions(context_50, num_questions=5)
generated_question_50 = []

for i, q in enumerate(questions_50, 1):
    generated_question_50.append(q)
    print(f"{i}. {q}")

1. Apa yang dianggap sebagai penyebab konflik militer Roma?
2. Apa yang membuat orang-orang Roma tidak mampu mempertahankan diri?
3. Apa alasan yang digunakan untuk konflik militer Roma?
4. Apa jenis pasukan Roma yang berperang dengan apa yang telah terjadi?
5. Apa alasan mengapa Roma dapat dimotivasi oleh imperialisme?


In [109]:
for i, q in enumerate(generated_question_50, 1):
    scores = scorer.score(reference_question_50, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.125, recall=0.16666666666666666, fmeasure=0.14285714285714288), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.125, recall=0.16666666666666666, fmeasure=0.14285714285714288)}
2. {'rouge1': Score(precision=0.1, recall=0.16666666666666666, fmeasure=0.125), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.1, recall=0.16666666666666666, fmeasure=0.125)}
3. {'rouge1': Score(precision=0.125, recall=0.16666666666666666, fmeasure=0.14285714285714288), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.125, recall=0.16666666666666666, fmeasure=0.14285714285714288)}
4. {'rouge1': Score(precision=0.09090909090909091, recall=0.16666666666666666, fmeasure=0.11764705882352942), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.09090909090909091, recall=0.16666666666666666, fmeasure=0.11764705882352942)}
5. {'rouge1': Score(

### Data Train 5

In [110]:
sample_data = raw_dataset['train'][5]
context_5 = sample_data['context']
reference_question_5 = sample_data['question']

print("Konteks:\n", context_5)
print("\nPertanyaan:\n", reference_question_5)

Konteks:
 Paket - paket buatan tangan hanya terdiri dari satu PID byte, dan umumnya dikirim sebagai tanggapan terhadap paket data. Deteksi galat disediakan dengan mentransmisikan empat bit yang mewakili tipe paket dua kali, dalam satu PID byte menggunakan formulir komplemen. Tiga tipe dasar adalah ACK, yang menunjukkan bahwa data berhasil diterima, NAK, menunjukkan bahwa data tidak dapat diterima dan harus dicoba ulang, dan STALL, menunjukkan bahwa perangkat tersebut memiliki kondisi galat dan tidak dapat mentransfer data sampai beberapa aksi perbaikan (seperti perangkat inisialisasi) terjadi.

Pertanyaan:
 Apa yang disediakan dengan transmisi empat bit yang mewakili tipe paket dua kali, dalam satu PID byte menggunakan bentuk komplemen?


In [111]:
questions_5 = generate_questions(context_5, num_questions=5)
generated_question_5 = []

for i, q in enumerate(questions_5, 1):
    generated_question_5.append(q)
    print(f"{i}. {q}")

1. Apa penjelasan dari CTG yang dibuat dengan menggunakan kebotan dan defisile?
2. Pada suatu pada waktu apa perangkat inisialisasi?
3. Apa yang membuat dua tipe, dan berapa banyak orang - orang yang mengalami perbaikan?
4. Apa jenis paket buatan tangan yang digunakan untuk mengirim data?
5. Apa bentuk paket yang digunakan dalam paket mesin buatan tangan?


In [112]:
for i, q in enumerate(generated_question_5, 1):
    scores = scorer.score(reference_question_5, q)
    print(f"{i}. {scores}")

1. {'rouge1': Score(precision=0.36363636363636365, recall=0.2, fmeasure=0.25806451612903225), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.36363636363636365, recall=0.2, fmeasure=0.25806451612903225)}
2. {'rouge1': Score(precision=0.14285714285714285, recall=0.05, fmeasure=0.07407407407407408), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.14285714285714285, recall=0.05, fmeasure=0.07407407407407408)}
3. {'rouge1': Score(precision=0.38461538461538464, recall=0.25, fmeasure=0.30303030303030304), 'rouge2': Score(precision=0.08333333333333333, recall=0.05263157894736842, fmeasure=0.06451612903225808), 'rougeL': Score(precision=0.23076923076923078, recall=0.15, fmeasure=0.18181818181818185)}
4. {'rouge1': Score(precision=0.3, recall=0.15, fmeasure=0.2), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.2, recall=0.1, fmeasure=0.13333333333333333)}
5. {'rouge1': Score(precisi

## Save Pretrained Model and Tokenizer

In [81]:
t5_model.save_pretrained("qg_model")
tokenizer.save_pretrained("qg_model")

('qg_model/tokenizer_config.json',
 'qg_model/special_tokens_map.json',
 'qg_model/spiece.model',
 'qg_model/added_tokens.json')